In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import pandas as pd

In [10]:
import sys
sys.path.append("..")

In [69]:
df = pd.read_parquet("../data/test.parquet")

In [70]:
%%time
vocabulary = set([x for xs in df['elements_1'] for x in xs])

CPU times: user 555 ms, sys: 14.8 ms, total: 570 ms
Wall time: 571 ms


# check the intersection

## brute force

In [71]:
from fast_filters.brute_force import brute_force_not_null_intersection

In [72]:
%%time
index_brute = brute_force_not_null_intersection(df, 'elements_1', 'elements_2')

CPU times: user 10.3 s, sys: 103 ms, total: 10.4 s
Wall time: 10.5 s


## binary

In [88]:
from fast_filters.binary import Converter, vectorized_not_null_intersection, vectorized_c2_contains_c1

In [74]:
%%time
converter = Converter(vocabulary=vocabulary)
df['elements_1_bin'] = df['elements_1'].map(converter.convert)
df['elements_2_bin'] = df['elements_2'].map(converter.convert)
index_vec = vectorized_not_null_intersection(df, 'elements_1_bin', 'elements_2_bin')

CPU times: user 3.99 s, sys: 77.8 ms, total: 4.07 s
Wall time: 4.15 s


In [90]:
%%time
index_vec = vectorized_not_null_intersection(df, 'elements_1_bin', 'elements_2_bin')

CPU times: user 5.39 ms, sys: 3.22 ms, total: 8.61 ms
Wall time: 6.98 ms


In [91]:
%%time
index_vec = vectorized_c2_contains_c1(df, 'elements_1_bin', 'elements_2_bin')

CPU times: user 5.52 ms, sys: 2.6 ms, total: 8.12 ms
Wall time: 6.25 ms


# checks

In [33]:
all(index_brute == index_vec)

True